# HairChangerAI_Training.ipynb

# Section 1: Setup and Installations

In [ ]:
!pip install diffusers==0.24.0 transformers accelerate safetensors
!pip install kaggle
!pip install opencv-python

In [ ]:
# Section 2: Kaggle Dataset Download
import os
os.environ['KAGGLE_USERNAME'] = 'Chinsa'  
os.environ['KAGGLE_KEY'] = 'd0f2c4a1b5e3b8f7c9d6a1e2c3e4f5a6'
  

In [ ]:

!kaggle datasets download -d kavyasreeb/hair-type-dataset
!unzip -q hair-type-dataset.zip -d hair_dataset

In [ ]:
# Section 3: Visualize the Dataset
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
data_path = "hair_dataset/Hair dataset/Hair dataset"
categories = os.listdir(data_path)

plt.figure(figsize=(12, 6))
for i, category in enumerate(categories[:5]):
    img_path = os.path.join(data_path, category, os.listdir(os.path.join(data_path, category))[0])
    img = cv2.imread(img_path)[..., ::-1]
    plt.subplot(1, 5, i + 1)
    plt.imshow(img)
    plt.title(category)
    plt.axis("off")
plt.show()

In [ ]:
# Section 4: Setup Fine-Tuning Pipeline
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch


In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    revision="fp16",
    use_auth_token=True  
).to("cuda")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

class HairDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = []
        self.prompts = []
        self.transform = transform

        for style in os.listdir(root_dir):
            style_path = os.path.join(root_dir, style)
            for img_name in os.listdir(style_path):
                img_path = os.path.join(style_path, img_name)
                self.image_paths.append(img_path)
                self.prompts.append(f"a person with {style.lower()} hair")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        prompt = self.prompts[idx]
        if self.transform:
            image = self.transform(image)
        return {"image": image, "prompt": prompt}

transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = HairDataset(data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
# Section 6: Fine-tune the Text Encoder + UNet (DreamBooth Style)
from diffusers import DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import UNet2DConditionModel, AutoencoderKL
from accelerate import Accelerator


print("✅ Dataset ready. From here, continue DreamBooth-style fine-tuning or LoRA...")


In [ ]:
# Section 7: Save the Fine-tuned Model
def save_model(pipe, save_path="hair_model"):
    pipe.save_pretrained(save_path)
    print(f"✅ Model saved to {save_path}")

save_model(pipe)



In [ ]:
# Section 8: Load the Fine-tuned Model
def load_model(model_path="hair_model"):
    pipe = StableDiffusionPipeline.from_pretrained(
        model_path,
        torch_dtype=torch.float16
    ).to("cuda")
    print(f"✅ Model loaded from {model_path}")
    return pipe

loaded_pipe = load_model()

# Section 9: Generate Images with the Fine-tuned Model